In [1]:
import pandas as pd
import numpy as np
import warnings
import pyodbc
import random
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

warnings.filterwarnings(action = 'ignore')

#### 유전 알고리즘 활용한 페어 트레이딩 v3

* v2
 - parameter 생성 시 공적분 검정 테스트를 추가하자 (train set 한도내에서)
 - child 생성 시 볼린저밴드 파라미터와 비중 2개로 나누어 crossover
 - mutation 시 볼린저밴드 파라미터 조정과 비중 shuffle

* v3
 - train set/test set 기간을 여러개로 나눠서 각 시뮬레이션 내 종목 비중과 결과 비교
 - 변이 이후 수선 기능을 추가. 각 파라미터의 정상성 테스트 후 p-value 0.05 이하인 것만 고르고 모자라다면 새로 생성

In [2]:
conn_pcor = pyodbc.connect('driver={Oracle in OraClient18Home1};dbq=PCOR;uid=EF0SEL;pwd=EF0SEL#076')
conn_quant = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=quant;uid=index;pwd=samsung@00')
conn_wisefn = pyodbc.connect('driver={SQL Server};server=46.2.90.172;database=wisefn;uid=index;pwd=samsung@00')

In [3]:
os.chdir("..")
os.chdir("..")

os.chdir('00_data')

#df_const = pd.read_json('230927_index_constituent.json')
df_prc_raw = pd.read_json('230927_stk_prc_daily.json')
#df_turnover = pd.read_json('230927_stk_turnover.json')
#df_sector = pd.read_json('230927_stk_sector.json')
#df_mktcap = pd.read_json('230927_stk_mktcap.json')
#df_turnover_daily = pd.read_json('231011_stk_turnover_daily.json')

os.chdir("..")
os.chdir('02_Trading Strategies')
os.chdir('231026_유전 알고리즘을 활용한 페어트레이딩')

In [4]:
sector_code = 'IKS013'

In [5]:
sql_dt = f'''
        SELECT TRD_DT, CLOSE_PRC
        FROM TS_IDX_DAILY
        WHERE 1=1
        AND SEC_CD = '{sector_code}'
        AND TRD_DT > '20091231'
        ORDER BY TRD_DT ASC
'''

df_dt = pd.read_sql(sql_dt, conn_wisefn)

In [6]:
df_dt['TRD_DT'] =  pd.to_datetime(df_dt['TRD_DT'])
df_dt['M'] = df_dt['TRD_DT'].dt.to_period('M')
dt_m = [max(df_dt[df_dt['M']==p]['TRD_DT']) for p in df_dt['M'].unique()]
dt_m = [int(x.strftime('%Y%m%d')) for x in dt_m]

In [12]:
def objective_func(df, params):
    
    #stk_list = sorted(list(set(df['COMP_CD'])))
    
    params_period = params[0]
    params_init = params[1]
    params_exit = params[2]
    params_wt = params[3:]

    df_sp = df * params_wt
    df_sp = df_sp.sum(axis=1)
    
    df_sp_norm = (df_sp - df_sp.rolling(params_period).mean())/df_sp.rolling(params_period).std()
    df_sp_norm.dropna(axis=0, inplace=True)
    
    init_dt = []
    exit_dt = []
    init_yn = 0
    for i in range(len(df_sp_norm)):
        if init_yn == 0:
            if df_sp_norm.iloc[i] < params_init:
                init_dt.append(df_sp_norm.index[i])
                init_yn = 1
        
        elif init_yn == 1:
            if df_sp_norm.iloc[i] > params_exit:
                exit_dt.append(df_sp_norm.index[i])
                init_yn = 0
        
    if len(init_dt) > len(exit_dt):
        if len(init_dt) - len(exit_dt) > 1:
            print("error")
        else:
            init_dt = init_dt[:-1]
    
    ret_list = []
    cum_ret = 1
    for i in range(len(init_dt)):
        dt_1 = init_dt[i]
        dt_2 = exit_dt[i]
        
        prc_1 = df.loc[dt_1]
        prc_2 = df.loc[dt_2]
        
        ret = prc_2/prc_1 - 1
        ret = (ret * params_wt).sum()
        ret_list.append(ret)
        cum_ret *= (1+ret)
    
    return cum_ret

In [13]:
def generate_weight(length):
    
    pos_len = random.randint(1,length - 1)
    neg_len = length - pos_len
    
    pos_wt = []
    for i in range(pos_len):
        p = random.uniform(0,1)
        pos_wt.append(p)
    
    pos_wt = [x/sum(pos_wt) for x in pos_wt]
    
    neg_wt = []
    for i in range(neg_len):
        n = random.uniform(0,1)
        neg_wt.append(n)
    
    neg_wt = [-x/sum(neg_wt) for x in neg_wt]
    
    wt = pos_wt + neg_wt
    
    random.shuffle(wt)
    
    return wt  
        

def generate_params(df, length):
    
    params_period = random.randint(20,20*36)
    params_init = random.uniform(-4,0)
    params_exit = random.uniform(0,4)
    
    while True:
        
        params_wt = generate_weight(length)
        
        df_sp = df * params_wt
        df_sp = df_sp.sum(axis=1)

        result = adfuller(df_sp)
        p_val = result[1]
        
        if p_val < 0.05:        
            params = [params_period, params_init, params_exit, *params_wt]
            
            break

            
    return params
    

In [14]:
def generate_pop(df, size, length):
    population = []
    
    for i in range(size):
        
        params = generate_params(df,length)
        population.append(params)
        
    return population

In [15]:
def compute_performance(df, population):
    
    obj_list = []
    for individual in population:
        obj = objective_func(df, individual)
        obj_list.append([individual,obj])
    
    pop_sorted = sorted(obj_list, key=lambda x: x[1], reverse=True)
    
    return pop_sorted

In [16]:
def select_survivors(population_sorted, best_sample, lucky_few, length, df):
    
    next_gen = []
    
    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_gen.append(population_sorted[i][0])
    
    lucky_index = np.random.choice(list(range(len(population_sorted))), lucky_few, replace=False)
    for i in lucky_index:
        next_gen.append(population_sorted[i][0])
    
    next_gen_2 = []
    for item in next_gen:
        if item not in next_gen_2:
            next_gen_2.append(item)
    
    
    while len(next_gen_2) < best_sample + lucky_few:
        next_gen_2.append(generate_params(df, length))
        
    random.shuffle(next_gen_2)
    
    return next_gen_2

In [17]:
def create_child(param1, param2):
    
    r1 = random.random()
    r2 = random.random()
    r3 = random.random()
    
    if r1 < 0.5:
        child = param1[:3] + param2[3:]
    else:
        child = param2[:3] + param1[3:]
            
    return child


def create_children(df, population, n_child):
    
    k = 1
    children = []
    
    fitness = [objective_func(df, x) for x in population]
    fitness_2 = [x if x >= 0 else 0 for x in fitness]
    
    prob = [x/sum(fitness_2) for x in fitness_2]
    

    while k < n_child:
        try:
            sample = np.random.choice(list(range(len(prob))), 2, replace=False, p = prob)
        except:
            sample = np.random.choice(list(range(len(prob))), 2, replace=False)
        parents = [population[sample[0]], population[sample[1]]]
        
        child = create_child(parents[0], parents[1])
        children.append(child)
        
        k += 1
    
    return children

In [18]:
def mutation1(param, prob):
    
    r1 = random.random()
    r2 = random.random()
    r3 = random.random()
    
     
    if r1 < prob:
        param_period = random.randint(20,20*36)
    else:
        param_period = param[0]
           
    if r2 < prob:
        param_init = random.uniform(-4,0)
    else:
        param_init = param[1]
        
    if r3 < prob:
        param_exit = random.uniform(0,4)
    else:
        param_exit = param[2]
        
        
    param_mutate = [param_period, param_init, param_exit, *param[3:]]
    
    return param_mutate


def mutation2(param, prob):
    
    r1 = random.random()
    
    param_wt = param[3:]
    
    if r1 < prob:
        random.shuffle(param_wt)

    param_mutate = param[:3] + param_wt
    
    return param_mutate



################################################
def mutate_pop(population, prob1, prob2):
    
    for i in range(len(population)):
            population[i] = mutation1(population[i], prob1)
            population[i] = mutation2(population[i], prob2)
    
    return population

In [19]:
def mutation_fix(population, df, length):
    
    population_fixed = []
    
    for i in population:
        
        params_wt = i[3:]
        
        df_sp = df * params_wt
        df_sp = df_sp.sum(axis=1)
        
        result = adfuller(df_sp)
        p_val = result[1]
        
        if p_val < 0.05:        
            individual_fixed = i
        else:
            new_wt = generate_params(df, length)[3:]
            individual_fixed = i[:3] + new_wt
                
        population_fixed.append(individual_fixed)
        
    return population_fixed


In [20]:
def simulation_func(df, params):
    
    #stk_list = sorted(list(set(df['COMP_CD'])))
    
    params_period = params[0]
    params_init = params[1]
    params_exit = params[2]
    params_wt = params[3:]

    df_sp = df * params_wt
    df_sp = df_sp.sum(axis=1)
    
    df_sp_norm = (df_sp - df_sp.rolling(params_period).mean())/df_sp.rolling(params_period).std()
    df_sp_norm.dropna(axis=0, inplace=True)
    
    init_dt = []
    exit_dt = []
    init_yn = 0
    for i in range(len(df_sp_norm)):
        if init_yn == 0:
            if df_sp_norm.iloc[i] < params_init:
                init_dt.append(df_sp_norm.index[i])
                init_yn = 1
        
        elif init_yn == 1:
            if df_sp_norm.iloc[i] > params_exit:
                exit_dt.append(df_sp_norm.index[i])
                init_yn = 0
        
    if len(init_dt) > len(exit_dt):
        if len(init_dt) - len(exit_dt) > 1:
            print("error")
        else:
            init_dt = init_dt[:-1]
    
    ret_list = []
    long_ret_list = []
    short_ret_list = []
    cum_ret = 1
    for i in range(len(init_dt)):
        dt_1 = init_dt[i]
        dt_2 = exit_dt[i]
        
        prc_1 = df.loc[dt_1]
        prc_2 = df.loc[dt_2]

        
        ret = prc_2/prc_1 - 1
        ret_ls = (ret * params_wt).sum()
        ret_list.append(ret_ls)        
        cum_ret *= (1+ret_ls)
        
        long_wt = [x if x >= 0 else 0 for x in params_wt]
        long_ret = (ret * long_wt).sum()
        long_ret_list.append(long_ret)
        
        short_wt = [x if x < 0 else 0 for x in params_wt]
        short_ret = (ret * short_wt).sum()
        short_ret_list.append(short_ret)
        
        
    
    return cum_ret, init_dt, exit_dt, ret_list, long_ret_list, short_ret_list

In [28]:
train_pct = [x/20 for x in range(10,17)]
sector_code = 'IKS013'

params_list = []
train_result_list = []
test_result_list = []
train_ret_list = []
test_ret_list = []



for i in train_pct:
    
    print(f'Train set percentage: {i}')
    
    
    base_d = dt_m[int(np.round(len(dt_m)*i,0))]
    train_begin_dt = dt_m[0]
    train_end_dt = base_d

    sql_const = f'''
                SELECT TRD_DT, CONCAT('A',STK_CD) COMP_CD, STK_NM_KOR COMP_NM
                FROM TS_STK_ISSUE
                WHERE 1=1
                AND KS200_TYP = 1
                AND TRD_DT = '{base_d}'
                AND KSC_CD = '{sector_code}'
    '''

    df_const = pd.read_sql(sql_const, conn_wisefn)

    df_train = df_prc_raw.copy()
    df_train = df_train[df_train['TRD_DT'] >= train_begin_dt]
    df_train = df_train[df_train['TRD_DT'] <= train_end_dt]
    df_train = df_train[df_train['COMP_CD'].isin(df_const['COMP_CD'])]
    df_train = df_train.sort_values(by = ['TRD_DT','COMP_CD'])
    #stk_list = sorted(list(set(df_train['COMP_CD'])))


    df_train = df_train.pivot(index = 'TRD_DT', columns = 'COMP_CD')
    df_train = df_train.droplevel(axis = 1, level = 0)
    #df_train = df_train[stk_list]
    df_train.dropna(axis=1, inplace=True)
    stk_list = sorted(list(df_train.columns))


    df_test = df_prc_raw.copy()
    df_test = df_test[df_test['TRD_DT'] >= train_end_dt]
    #df_test = df_prc[df_prc['TRD_DT'] <= train_end_dt]
    df_test = df_test[df_test['COMP_CD'].isin(df_const['COMP_CD'])]
    df_test = df_test.sort_values(by = ['TRD_DT','COMP_CD'])

    df_test = df_test.pivot(index = 'TRD_DT', columns = 'COMP_CD')
    df_test = df_test.droplevel(axis = 1, level = 0)
    df_test = df_test[stk_list]
    
    print(f'  Train set: {train_begin_dt} ~ {train_end_dt}')
    print(f'  Test set: {train_end_dt} ~ {max(df_test.index)}')
    
    print(f'  유전 알고리즘 시작')
    n_generation = 200
    population = 300
    best_sample = int(population/3)
    lucky_few = int(population/3)
    prob1 = 0.05
    prob2 = 0.075
    param_length = len(stk_list)
    
    pop = generate_pop(df_train, size = population, length = param_length)

    best_gene = []
    best_perf = []

    for g in range(n_generation):

        pop_sorted = compute_performance(df_train, pop)

        survivors = select_survivors(pop_sorted, best_sample, lucky_few, param_length, df_train)

        children = create_children(df_train, survivors, population)

        new_generation = mutate_pop(children, prob1, prob2)
        
        new_generation = mutation_fix(new_generation, df_train, param_length)

        pop = new_generation
        best_gene.append(pop_sorted[0][0])
        best_perf.append(pop_sorted[0][1])
        
        temp_sp = df_train * pop_sorted[0][0][3:]
        temp_sp = temp_sp.sum(axis=1)
        result = adfuller(temp_sp)
        p_val = result[1]

        print(f'====== {g}th generation ends ======')
        print(pop_sorted[0])
        print(f'정상성 체크: p value = {p_val}')
    
    temp_params = pd.DataFrame(pop_sorted[0][0], columns = [i], index = ['period','init_sd','exit_sd',*stk_list]).T
    
    result_train = simulation_func(df_train,pop_sorted[0][0])
    result_test = simulation_func(df_test,pop_sorted[0][0])
    
    ret_train = pd.DataFrame([result_train[3],result_train[4],result_train[5]], index = ['ls','long','short']).T
    ret_test = pd.DataFrame([result_test[3],result_test[4],result_test[5]], index = ['ls','long','short']).T
    
    
    params_list.append(temp_params)
    train_result_list.append([result_train[0],(ret_train > 0).astype(int).sum()['ls']/len(ret_train),len(ret_train)])
    test_result_list.append([result_test[0], (ret_test > 0).astype(int).sum()['ls']/len(ret_test), len(ret_test)])
    
    train_ret_list.append(ret_train)
    test_ret_list.append(ret_test)

Train set percentage: 0.5
  Train set: 20100129 ~ 20170131
  Test set: 20170131 ~ 20230927
  유전 알고리즘 시작
====== 0th generation ends ======
[[404, -0.7296950118349277, 2.31270286764133, 0.08257314462217603, 0.1571693696280877, 0.0036877771051546616, 0.1971111631169836, 0.01952582180595248, 0.10331497711242571, 0.008695838722180684, 0.07280822616174532, -1.0, 0.11506837976114856, 0.03898103470682728, 0.003504002550176791, 0.1325137991719093, 0.06504646553523181], 2.7805524787253213]
정상성 체크: p value = 0.021848874880304965
====== 1th generation ends ======
[[361, -0.2959498083674199, 0.44463559352841564, 0.08257314462217603, 0.1571693696280877, 0.0036877771051546616, 0.1971111631169836, 0.01952582180595248, 0.10331497711242571, 0.008695838722180684, 0.07280822616174532, -1.0, 0.11506837976114856, 0.03898103470682728, 0.003504002550176791, 0.1325137991719093, 0.06504646553523181], 2.868878923889836]
정상성 체크: p value = 0.021848874880304965
====== 2th generation ends ======
[[174, -0.0606772633

====== 20th generation ends ======
[[160, -0.4120881877917353, 1.9367461449012335, 0.08257314462217603, 0.1571693696280877, 0.0036877771051546616, 0.1971111631169836, 0.01952582180595248, 0.10331497711242571, 0.008695838722180684, 0.07280822616174532, -1.0, 0.11506837976114856, 0.03898103470682728, 0.003504002550176791, 0.1325137991719093, 0.06504646553523181], 5.609522662659067]
정상성 체크: p value = 0.021848874880304965
====== 21th generation ends ======
[[160, -0.4120881877917353, 1.9367461449012335, 0.08257314462217603, 0.1571693696280877, 0.0036877771051546616, 0.1971111631169836, 0.01952582180595248, 0.10331497711242571, 0.008695838722180684, 0.07280822616174532, -1.0, 0.11506837976114856, 0.03898103470682728, 0.003504002550176791, 0.1325137991719093, 0.06504646553523181], 5.609522662659067]
정상성 체크: p value = 0.021848874880304965
====== 22th generation ends ======
[[150, -0.8003615094338765, 2.1545426307745625, 0.052566060726450335, 0.11821205765481331, 0.016124395393699132, 0.024314

====== 40th generation ends ======
[[160, -0.4120881877917353, 1.9367461449012335, 0.08257314462217603, 0.1571693696280877, 0.0036877771051546616, 0.1971111631169836, 0.01952582180595248, 0.10331497711242571, 0.008695838722180684, 0.07280822616174532, -1.0, 0.11506837976114856, 0.03898103470682728, 0.003504002550176791, 0.1325137991719093, 0.06504646553523181], 5.609522662659067]
정상성 체크: p value = 0.021848874880304965
====== 41th generation ends ======
[[160, -1.1493502565440732, 2.0570871980088152, 0.0736229809335735, 0.12560194310343167, 0.06621838860529294, 0.01988868637097855, 0.07794758627037167, 0.15357732085797782, 0.021124096385695967, 0.09043470965806052, -1.0, 0.12382322867860004, 0.016459994822436548, 0.08051332699232411, 0.06876007857037175, 0.08202765875088495], 5.351880575450341]
정상성 체크: p value = 0.004614965195244772
====== 42th generation ends ======
[[170, -0.9724500662920845, 1.7619164960296092, 0.1281169765845418, 0.047229920777220956, 0.04275960223385486, 0.11937748

====== 60th generation ends ======
[[170, -0.9724500662920845, 1.7619164960296092, 0.052566060726450335, 0.11821205765481331, 0.016124395393699132, 0.024314840236887587, 0.11644159011696434, 0.08447680603941679, 0.05105406517714992, 0.11908099144339819, -1.0, 0.11615628708353384, 0.04675282260011784, 0.06679681702005172, 0.08161693642336262, 0.10640633008415423], 5.704236042008023]
정상성 체크: p value = 0.022448405426686538
====== 61th generation ends ======
[[164, -1.2050986104208068, 1.8722260050245128, 0.0736229809335735, 0.12560194310343167, 0.06621838860529294, 0.01988868637097855, 0.07794758627037167, 0.15357732085797782, 0.021124096385695967, 0.09043470965806052, -1.0, 0.12382322867860004, 0.016459994822436548, 0.08051332699232411, 0.06876007857037175, 0.08202765875088495], 5.9258708559208495]
정상성 체크: p value = 0.004614965195244772
====== 62th generation ends ======
[[164, -1.2050986104208068, 1.8722260050245128, 0.0736229809335735, 0.12560194310343167, 0.06621838860529294, 0.019888

====== 80th generation ends ======
[[164, -1.2050986104208068, 2.148370693437245, 0.11937748028646762, 0.1281169765845418, 0.04275960223385486, 0.12112680155229759, 0.05117741001700676, 0.147270335783562, 0.023708790383374066, 0.094144727220323, -1.0, 0.10728137243135281, 0.047229920777220956, 0.07836536147514604, 0.012407541694263448, 0.027033679560588917], 6.2290979522794485]
정상성 체크: p value = 0.019001285384743112
====== 81th generation ends ======
[[164, -1.0119894548023027, 1.9066790402042941, 0.07797683798559579, 0.11839160535162331, 0.045876908194006735, 0.1058158550907874, 0.10365455942070803, 0.09648506725253515, 0.01869084521464395, 0.09985628279087248, -1.0, 0.09948190617818325, 0.029094698771651273, 0.09155878662305904, 0.05695873104688267, 0.056157916079450844], 6.226431398531682]
정상성 체크: p value = 0.026587249782095573
====== 82th generation ends ======
[[164, -1.0119894548023027, 1.9066790402042941, 0.07797683798559579, 0.11839160535162331, 0.045876908194006735, 0.10581585

...

====== 9th generation ends ======
[[178, -2.2011708598076907, 1.6277421010110262, 0.15414189739876583, 0.0021392856696714215, 0.10782706592965934, 0.05328488335235595, 0.19133264146709472, 0.1655254603482119, 0.11850035142517579, 0.06654356480969102, 0.07198980951638405, 0.007797437320697708, 0.06091760276229239, -1.0], 5.004864102540919]
정상성 체크: p value = 0.04743200717262472
====== 10th generation ends ======
[[178, -2.2011708598076907, 1.6277421010110262, 0.10332899002138765, 0.21678140919621336, 0.08569213488396517, 0.09798951136891942, 0.15565285992340158, 0.0034927267799764886, 0.18230621205136915, 0.10140402877311562, 0.01108753483412715, 0.02631433653805352, 0.01595025562947073, -1.0], 4.065488888114353]
정상성 체크: p value = 0.042976056127604245
====== 11th generation ends ======
[[178, -2.2011708598076907, 1.6277421010110262, 0.15414189739876583, 0.0021392856696714215, 0.10782706592965934, 0.05328488335235595, 0.19133264146709472, 0.1655254603482119, 0.11850035142517579, 0.0665435

====== 31th generation ends ======
[[178, -2.2818029709381333, 2.052355066706599, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.027395769868630813, -0.242095322902701, -0.03959899498223363, -0.006279875865912566, -0.11972532780945869, -0.0025449532923250313, -0.2660485304924257], 7.5279502026007075]
정상성 체크: p value = 0.01880739752819855
====== 32th generation ends ======
[[178, -2.344044519900933, 2.052355066706599, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.027395769868630813, -0.242095322902701, -0.03959899498223363, -0.006279875865912566, -0.11972532780945869, -0.0025449532923250313, -0.2660485304924257], 7.5279502026007075]
정상성 체크: p value = 0.01880739752819855
====== 33th generation ends ======
[[178, -2.2818029709381333, 2.052355066706599, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.

====== 52th generation ends ======
[[602, -1.3844044035000964, 0.4905675335621842, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.027395769868630813, -0.242095322902701, -0.03959899498223363, -0.006279875865912566, -0.11972532780945869, -0.0025449532923250313, -0.2660485304924257], 9.980960503346083]
정상성 체크: p value = 0.01880739752819855
====== 53th generation ends ======
[[602, -1.3844044035000964, 0.4905675335621842, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.027395769868630813, -0.242095322902701, -0.03959899498223363, -0.006279875865912566, -0.11972532780945869, -0.0025449532923250313, -0.2660485304924257], 9.980960503346083]
정상성 체크: p value = 0.01880739752819855
====== 54th generation ends ======
[[602, -1.3844044035000964, 0.4905675335621842, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -

====== 73th generation ends ======
[[602, -1.3844044035000964, 0.4905675335621842, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.027395769868630813, -0.242095322902701, -0.03959899498223363, -0.006279875865912566, -0.11972532780945869, -0.0025449532923250313, -0.2660485304924257], 9.980960503346083]
정상성 체크: p value = 0.01880739752819855
====== 74th generation ends ======
[[602, -1.3844044035000964, 0.4905675335621842, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -0.027395769868630813, -0.242095322902701, -0.03959899498223363, -0.006279875865912566, -0.11972532780945869, -0.0025449532923250313, -0.2660485304924257], 9.980960503346083]
정상성 체크: p value = 0.01880739752819855
====== 75th generation ends ======
[[602, -1.3844044035000964, 0.4905675335621842, -0.014375190525440043, 0.7216517180384977, -0.027713795341632902, -0.25422223891923973, 0.2783482819615023, -

KeyboardInterrupt: 

In [35]:
temp_params = pd.DataFrame(pop_sorted[0][0], columns = [i], index = ['period','init_sd','exit_sd',*stk_list]).T
    
result_train = simulation_func(df_train,pop_sorted[0][0])
result_test = simulation_func(df_test,pop_sorted[0][0])

ret_train = pd.DataFrame([result_train[3],result_train[4],result_train[5]], index = ['ls','long','short']).T
ret_test = pd.DataFrame([result_test[3],result_test[4],result_test[5]], index = ['ls','long','short']).T


params_list.append(temp_params)
train_result_list.append([result_train[0],(ret_train > 0).astype(int).sum()['ls']/len(ret_train),len(ret_train)])
test_result_list.append([result_test[0], (ret_test > 0).astype(int).sum()['ls']/len(ret_test), len(ret_test)])

train_ret_list.append(ret_train)
test_ret_list.append(ret_test)

In [30]:
df_params = params_list[0].T
for i in range(1, len(params_list)):
    df_params = df_params.merge(params_list[i].T, how = 'outer', left_index = True, right_index = True)

df_params.fillna(0, inplace=True)
    
df_train_result = pd.DataFrame(train_result_list, index = train_pct[:4], columns = ['cum_ret','hit_ratio','cnt'])
df_test_result = pd.DataFrame(test_result_list, index = train_pct[:4], columns = ['cum_ret','hit_ratio','cnt'])

In [31]:
df_params.round(3)

,0.50,0.55,0.60,0.65
A000660,0.156,-0.064,-0.145,-0.014
A000990,0.065,0.647,0.681,0.722
A004490,0.030,-0.084,-0.122,-0.028
A004710,0.041,-0.097,0.000,0.000
A005930,0.071,-0.062,-0.039,-0.254
A006400,0.084,0.353,0.319,0.278
A008060,0.091,-0.100,-0.117,-0.027
A009150,0.110,-0.030,-0.114,-0.242
A010120,-1.000,-0.107,-0.117,-0.040
A011070,0.172,-0.129,-0.029,-0.006


In [32]:
df_train_result

,cum_ret,hit_ratio,cnt
0.50,6.415994,1.000000,8
0.55,5.819400,0.750000,8
0.60,9.289565,0.846154,13
0.65,9.980961,1.000000,6


In [33]:
df_test_result

,cum_ret,hit_ratio,cnt
0.50,1.814053,1.00,4
0.55,1.805380,0.75,4
0.60,2.172412,1.00,4
0.65,1.000000,NaN,0
